In [7]:
import sys
sys.path.insert(0, '../')
import numpy as np
from qhdopt import QHD
from qhdopt.utils.benchmark_utils import run_test

In [8]:
from sympy import symbols
x, y = symbols('x y')
functions = [(-x**2 + x*y - 0.5 * y ** 2 + 3. / 4 * x - 1. / 4 * y, [x, y])]

<h3> List of parameters </>

In [9]:
backend = "qutip"
embedding_scheme = "onehot"
resolution = 6
penalty = 50
discretization = 40
shots = 1000
gamma = 30
tol = 1e-3
penalty_coefficient = 1.2

In [10]:
data = np.zeros((len(functions), 12))

In [11]:
for i in range(0, len(functions)):
    print(f"Test problem {i}:")
    function, syms = functions[i]
    model = QHD.SymPy(function, syms)
    model.qutip_setup(resolution,shots, embedding_scheme=embedding_scheme, penalty_coefficient=penalty_coefficient, time_discretization=discretization, gamma=gamma, post_processing_method="TNC")
    data[i] = run_test(model)

Test problem 0:
Compiled.
Solved.
* Coarse solution
Minimizer: [0.16666667 1.        ]
Affined Minimizer: [0.16666667 1.        ]
Minimum: -0.4861111

* Fine-tuned solution
Minimizer: [0. 1.]
Affined Minimizer: [0. 1.]
Minimum: -0.75
Success rate: 0.991

* Runtime breakdown
SimuQ compilation: 2.146 s
Backend runtime: 19.915 s
Decoding time: 0.247 s
Fine-tuning time: 0.480 s
* Total time: 22.789 s


In [6]:
x, y, z = symbols('x y z')
f = (y**2)*z - x*z + (x**2)*y
model = QHD.SymPy(f, [x, y, z])
model.qutip_setup(4, time_discretization=40)
model.optimize(verbose=1)

Compiled.
Solved.
* Coarse solution
Minimizer: [1.   0.25 1.  ]
Affined Minimizer: [1.   0.25 1.  ]
Minimum: -0.6875

* Fine-tuned solution
Minimizer: [1. 0. 1.]
Affined Minimizer: [1. 0. 1.]
Minimum: -1.0
Success rate: 1.0

* Runtime breakdown
SimuQ compilation: 2.137 s
Backend runtime: 8.236 s
Decoding time: 0.072 s
Fine-tuning time: 0.084 s
* Total time: 10.529 s


Array(-1., dtype=float32)